<a href="https://colab.research.google.com/github/jbrown544/interlingual-topic-modeling/blob/main/Interlingual_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Thematic Discovery in Non-English Natural Language Corpora

*Note: Use the "Open in Colab" link above to render inline visualizations.*

In [29]:
import warnings

# supress library maintenance warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [30]:
# update package management then install required versions of dependencies
!pip install -q -U pip setuptools wheel python-Levenshtein \
                   gensim==4.1.0 spacy==3.1.2 pyLDAvis==3.3.1

In [31]:
# inspect Gensim package information
!pip show gensim

Name: gensim
Version: 4.1.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, smart-open, scipy
Required-by: pyLDAvis


In [32]:
# inspect spaCy package information
!pip show spacy

Name: spacy
Version: 3.1.2
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: spacy-legacy, numpy, catalogue, requests, blis, typer, thinc, pathy, wasabi, jinja2, tqdm, pydantic, typing-extensions, setuptools, packaging, preshed, srsly, cymem, murmurhash
Required-by: fastai, en-core-web-sm


In [33]:
# instpect pyLDAvis package information
!pip show pyLDAvis

Name: pyLDAvis
Version: 3.3.1
Summary: Interactive topic model visualization. Port of the R package.
Home-page: https://github.com/bmabey/pyLDAvis
Author: Ben Mabey
Author-email: ben@benmabey.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: funcy, scikit-learn, scipy, sklearn, future, gensim, numpy, jinja2, setuptools, joblib, pandas, numexpr
Required-by: 




---



In [34]:
# import dependencies

from urllib.parse import urljoin

from smart_open import open

from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [35]:
# samples from the french language

# samples are accessible from within this repository
french_book_url = 'https://raw.githubusercontent.com/jbrown544/interlingual-topic-modeling/main/text/books/fr/'

jules_verne_french_books = [
  urljoin(french_book_url, 'Around_the_World_in_Eighty_Days.txt'),
  urljoin(french_book_url, 'Five_Weeks_in_a_Balloon.txt'),
  urljoin(french_book_url, 'From_the_Earth_to_the_Moon.txt'),
  urljoin(french_book_url, 'Robur_the_Conqueror.txt'),
  urljoin(french_book_url, 'The_Mutineers_of_the_Bounty.txt')
]

In [36]:
# prototype of text preprocessing pipeline
def normalize_text(text,
                   deaccent=False,
                   min_token_length=2,
                   max_token_length=15):
  
  # tokenizization, lowercasing, filtering tokens to length range,
  # and optionally removing accenting marks
  processed_text = simple_preprocess(text, deacc=deaccent,
                                     min_len=min_token_length,
                                     max_len=max_token_length)

  return processed_text

In [37]:
# read and process documents into processed lists
jules_verne_texts = \
  [normalize_text(open(book, encoding='latin-1').read()) \
   for book in jules_verne_french_books]

In [38]:
# construct dictionary from processed lists
jules_verne_dictionary = corpora.Dictionary(jules_verne_texts)

print(jules_verne_dictionary)

Dictionary(20937 unique tokens: ['abaissait', 'abaissant', 'abaissement', 'abaissé', 'abandonna']...)


In [39]:
# filter tokens not appearing in at least N documents
# or appearing in over % of documents
# jules_verne_dictionary.filter_extremes(no_below=1,
#                                        no_above=0.5)

# print(jules_verne_dictionary)

In [40]:
# construct corpus (document-term matrix) by converting 
# processed lists into bag-of-words representation
jules_verne_corpus = [jules_verne_dictionary.doc2bow(text) \
                      for text in jules_verne_texts]

In [41]:
# construct HDP model (topics discovered during processing)
jules_verne_hdp_model = models.HdpModel(jules_verne_corpus, 
                                        id2word=jules_verne_dictionary)

In [42]:
# enable visualizations inline
pyLDAvis.enable_notebook()

In [43]:
# visualize suggested LDA model derived from computed HDP model
gensimvis.prepare(jules_verne_hdp_model.suggested_lda_model(),
                  jules_verne_corpus, jules_verne_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster          Freq
topic                                                   
0     -0.000045 -0.000384       1        1  5.574190e+01
1     -0.000346  0.000175       2        1  2.161095e+01
2     -0.000182  0.000160       3        1  2.003258e+01
3      0.000096 -0.000252       4        1  2.614448e+00
4      0.000149  0.000285       5        1  6.128530e-05
...         ...       ...     ...      ...           ...
145   -0.000136  0.000160     146        1  2.198502e-47
146   -0.000071 -0.000087     147        1  1.099251e-47
147   -0.000003 -0.000195     148        1  5.496256e-48
148   -0.000029  0.000090     149        1  2.748128e-48
149    0.000077  0.000153     150        1  2.748128e-48

[150 rows x 5 columns], topic_info=                Term          Freq      Total  Category  logprob  loglift
13347       rapports  1.600000e+01  16.000000   Default  30.0000  30.0000
8325         théâtre  1.600000e+01  16.000000   Default  29.0000  29.0000
9986          bnrton  1.600000e+01  16.000000   Default  28.0000  28.0000
16064         farces  1.600000e+01  16.000000   Default  27.0000  27.0000
484    appartiennent  1.500000e+01  15.000000   Default  26.0000  26.0000
...              ...           ...        ...       ...      ...      ...
15273        caisson  4.674088e-49  12.845814  Topic150  -9.6588   0.2807
15320      cavalcade  4.673283e-49  12.938617  Topic150  -9.6589   0.2733
16510        joyeuse  4.655594e-49  12.767329  Topic150  -9.6627   0.2828
7919          sonnée  4.677884e-49  13.486150  Topic150  -9.6579   0.2328
15342       chantait  4.674275e-49  13.497812  Topic150  -9.6587   0.2312

[8320 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
3          1  0.571809   abaissé
3          2  0.245061   abaissé
3          3  0.245061   abaissé
9364       1  0.531499  abaissés
9364       2  0.177166  abaissés
...      ...       ...       ...
9359       2  0.175130      îles
9359       3  0.175130      îles
14853      1  0.575744     ôtant
14853      2  0.246748     ôtant
14853      3  0.164498     ôtant

[16932 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150])